# Smartphone motion detection using ANN
# Precision ~ 95%

#### This simple neural network was trained on the public dataset made by UCI ML:

https://www.kaggle.com/uciml/human-activity-recognition-with-smartphones 

#### Brief description from the data provider:

"The Human Activity Recognition database was built from the recordings of 30 study participants performing activities of daily living (ADL) while carrying a waist-mounted smartphone with embedded inertial sensors. The objective is to classify activities into one of the six activities performed.

The experiments have been carried out with a group of **30 volunteers within an age bracket of 19-48 years. Each person performed six activities (WALKING, WALKINGUPSTAIRS, WALKINGDOWNSTAIRS, SITTING, STANDING, LAYING) wearing a smartphone (Samsung Galaxy S II) on the waist.** Using its embedded accelerometer and gyroscope, we captured 3-axial linear acceleration and 3-axial angular velocity at a constant rate of 50Hz. The experiments have been video-recorded to label the data manually. The obtained dataset has been randomly partitioned into two sets, where 70% of the volunteers was selected for generating the training data and 30% the test data."

Neural network architecture:

 - 70% of data (train set) is split into train and test with 0.2 ratio
 - 30% of data (test set) is loaded as **validation set**
 - X sample has shape [1, 562], Standard-scaled
 - y is 563rd One Hot Encoded column, 
 - Keras start model is built and compiled with 1 layer and 6 neurons, then:
     - Wrapped in scikit-learn KerasClassifier wrapper
     - Cross-validated with Randomized Seach CV grid (15 min run on regular laptop)
 - Resulting model is saved in repository

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

## Import and preprocess data

In [2]:
df = pd.read_csv("/Users/a1/Desktop/Notebooks/Accelerometer/train.csv")
df.shape

(7352, 563)

In [3]:
df.head(10)

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING
5,0.277199,-0.010098,-0.105137,-0.997335,-0.990487,-0.995420,-0.997627,-0.990218,-0.995549,-0.942469,...,-0.844619,0.082632,-0.143439,0.275041,-0.368224,-0.849632,0.184823,-0.042126,1,STANDING
6,0.279454,-0.019641,-0.110022,-0.996921,-0.967186,-0.983118,-0.997003,-0.966097,-0.983116,-0.940987,...,-0.564430,-0.212754,-0.230622,0.014637,-0.189512,-0.852150,0.182170,-0.043010,1,STANDING
7,0.277432,-0.030488,-0.125360,-0.996559,-0.966728,-0.981585,-0.996485,-0.966313,-0.982982,-0.940987,...,-0.421715,-0.020888,0.593996,-0.561871,0.467383,-0.851017,0.183779,-0.041976,1,STANDING
8,0.277293,-0.021751,-0.120751,-0.997328,-0.961245,-0.983672,-0.997596,-0.957236,-0.984379,-0.940598,...,-0.572995,0.012954,0.080936,-0.234313,0.117797,-0.847971,0.188982,-0.037364,1,STANDING
9,0.280586,-0.009960,-0.106065,-0.994803,-0.972758,-0.986244,-0.995405,-0.973663,-0.985642,-0.940028,...,0.140452,-0.020590,-0.127730,-0.482871,-0.070670,-0.848294,0.190310,-0.034417,1,STANDING


In [4]:
# Shuffle original data to get rid of possible patterns

from sklearn.utils import shuffle
df = shuffle(df)
df.reset_index(inplace=True)
df.drop(columns='index', axis=1, inplace=True)

In [5]:
# Define X and y

X = df.iloc[:, 0:562]
y = df.iloc[:, -1:]

In [6]:
# Scale x

from sklearn.preprocessing import OneHotEncoder, StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)
X.shape

(7352, 562)

In [7]:
# Encode y

ohe = OneHotEncoder()
y_encoded = ohe.fit_transform(y)
y_encoded.shape

(7352, 6)

## Split to train / test

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
# Train/test size is 0.8 / 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2)
print("Train data shape: ", X_train.shape, y_train.shape, "\nTest data shape: ", X_test.shape, y_test.shape)

Train data shape:  (5881, 562) (5881, 6) 
Test data shape:  (1471, 562) (1471, 6)


In [10]:
# Check one hot encoded labels

ohe.categories_

[array(['LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS',
        'WALKING_UPSTAIRS'], dtype=object)]

## Validation set

In [11]:
# Load validation set

df2 = df = pd.read_csv("/Users/a1/Desktop/Notebooks/Accelerometer/test.csv")
df2.shape

(2947, 563)

In [12]:
# Create X_val, y_val; shuffle, scale, encode

df2 = shuffle(df2)
X_val = df2.iloc[:, 0:562]
y_val = df2.iloc[:, -1:]

X_val = scaler.fit_transform(X_val)
y_val_encoded = ohe.fit_transform(y_val)

## Keras Model

### Callbacks

In [14]:
# Early Stopping callback

early_stopping_callback = keras.callbacks.EarlyStopping(patience=15)

In [15]:
# Tensorflow callback

import os
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)


### Training and tuning

#### Randomized GridSearchCV using sklearn KerasClassifier wrapper

In [16]:
from keras.wrappers import scikit_learn
from sklearn.model_selection import RandomizedSearchCV

# Function

def build_model(n_layers=1, n_neurons=6, activation='elu', optimizer='Nadam'):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[562])),    # always specify "input_shape"
    for n in range(n_layers):
        model.add(keras.layers.Dense(n_neurons, kernel_initializer='he_normal', activation=activation)),

    model.add(keras.layers.Dense(6, kernel_initializer='he_normal', activation="softmax"))

    model.compile(loss="categorical_crossentropy", metrics="accuracy", optimizer=optimizer)
    
    return model

In [17]:
# These parameters are already pre-selected based on previous cross validations

params = {"n_layers": [1,2,3,],
         "n_neurons": [17,19],
         }

In [18]:
# Wrap function in scikit-learn

keras_clf = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [19]:
# Feeding grid of parameters to wrapper

rnd_clf = RandomizedSearchCV(keras_clf, param_distributions=params)

In [20]:
# Cross validating classifier - long run

rnd_clf.fit(X_train, y_train.toarray(), epochs=100,
            validation_data=(X_test, y_test.toarray()), 
            callbacks=[early_stopping_callback,  tensorboard_cb],)

/Users/a1/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Epoch 1/100
147/147 [==============================] - 1s 6ms/step - loss: 0.9844 - accuracy: 0.6411 - val_loss: 0.2310 - val_accuracy: 0.9300
Epoch 2/100
147/147 [==============================] - 0s 1ms/step - loss: 0.1792 - accuracy: 0.9429 - val_loss: 0.1323 - val_accuracy: 0.9545
Epoch 3/100
147/147 [==============================] - 0s 1ms/step - loss: 0.1238 - accuracy: 0.9597 - val_loss: 0.0989 - val_accuracy: 0.9646
Epoch 4/100
147/147 [==============================] - 0s 1ms/step - loss: 0.0824 - accuracy: 0.9769 - val_loss: 0.1087 - val_accuracy: 0.9599
Epoch 5/100
147/147 [==============================] - 0s 1ms/step - loss: 0.0720 - accuracy: 0.9782 - val_loss: 0.0754 - val_accuracy: 0.9735
Epoch 6/100
147/147 [==============================] - 0s 1ms/step - loss: 0.0604 - accuracy: 0.9813 - val_loss: 0.1346 - val_accuracy: 0.9456
Epoch 7/100
147/147 [==============================] - 0s 2ms/step - loss: 0.0620 - accuracy: 0.9766 - val_loss: 0.0723 - val_accuracy: 0.9755

148/148 [==============================] - 0s 1ms/step - loss: 0.2132 - accuracy: 0.9580 - val_loss: 0.0733 - val_accuracy: 0.9742
Epoch 18/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0241 - accuracy: 0.9936 - val_loss: 0.0520 - val_accuracy: 0.9782
Epoch 19/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0247 - accuracy: 0.9913 - val_loss: 0.0537 - val_accuracy: 0.9810
Epoch 20/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0179 - accuracy: 0.9938 - val_loss: 0.0513 - val_accuracy: 0.9810
Epoch 21/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0161 - accuracy: 0.9957 - val_loss: 0.0485 - val_accuracy: 0.9810
Epoch 22/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0164 - accuracy: 0.9952 - val_loss: 0.0490 - val_accuracy: 0.9816
Epoch 23/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0162 - accuracy: 0.9953 - val_loss: 0.0468 - val_accuracy: 0.9830
Epoch

148/148 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0469 - val_accuracy: 0.9823
Epoch 36/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0481 - val_accuracy: 0.9850
Epoch 37/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0515 - val_accuracy: 0.9803
Epoch 38/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0496 - val_accuracy: 0.9850
Epoch 39/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0513 - val_accuracy: 0.9830
Epoch 40/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0502 - val_accuracy: 0.9844
Epoch 41/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0509 - val_accuracy: 0.9816
Epoch

148/148 [==============================] - 0s 1ms/step - loss: 0.0089 - accuracy: 0.9974 - val_loss: 0.0583 - val_accuracy: 0.9762
Epoch 30/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0067 - accuracy: 0.9995 - val_loss: 0.0454 - val_accuracy: 0.9830
Epoch 31/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0059 - accuracy: 0.9993 - val_loss: 0.0537 - val_accuracy: 0.9823
Epoch 32/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0049 - accuracy: 0.9990 - val_loss: 0.0485 - val_accuracy: 0.9810
Epoch 33/100
148/148 [==============================] - 0s 2ms/step - loss: 0.0044 - accuracy: 0.9997 - val_loss: 0.0462 - val_accuracy: 0.9850
Epoch 34/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0056 - accuracy: 0.9998 - val_loss: 0.0564 - val_accuracy: 0.9782
Epoch 35/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0049 - accuracy: 0.9996 - val_loss: 0.0522 - val_accuracy: 0.9810
Epoch

...

Epoch 4/100
148/148 [==============================] - 0s 2ms/step - loss: 0.0803 - accuracy: 0.9744 - val_loss: 0.0992 - val_accuracy: 0.9606
Epoch 5/100
148/148 [==============================] - 0s 2ms/step - loss: 0.0753 - accuracy: 0.9684 - val_loss: 0.0694 - val_accuracy: 0.9728
Epoch 6/100
148/148 [==============================] - 0s 2ms/step - loss: 0.0507 - accuracy: 0.9825 - val_loss: 0.0640 - val_accuracy: 0.9721
Epoch 7/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0390 - accuracy: 0.9868 - val_loss: 0.0761 - val_accuracy: 0.9714
Epoch 8/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0395 - accuracy: 0.9843 - val_loss: 0.1008 - val_accuracy: 0.9613
Epoch 9/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0347 - accuracy: 0.9901 - val_loss: 0.0853 - val_accuracy: 0.9687
Epoch 10/100
148/148 [==============================] - 0s 2ms/step - loss: 0.0293 - accuracy: 0.9894 - val_loss: 0.0670 - val_accuracy: 0.974

147/147 [==============================] - 0s 1ms/step - loss: 6.4887e-04 - accuracy: 1.0000 - val_loss: 0.0679 - val_accuracy: 0.9816
Epoch 30/100
147/147 [==============================] - 0s 1ms/step - loss: 5.2224e-04 - accuracy: 1.0000 - val_loss: 0.0670 - val_accuracy: 0.9823
Epoch 31/100
147/147 [==============================] - 0s 1ms/step - loss: 5.0857e-04 - accuracy: 1.0000 - val_loss: 0.0675 - val_accuracy: 0.9803
Epoch 32/100
37/37 [==============================] - 0s 1ms/step - loss: 0.0415 - accuracy: 0.9864
Epoch 1/100
148/148 [==============================] - 1s 3ms/step - loss: 0.8909 - accuracy: 0.6859 - val_loss: 0.3694 - val_accuracy: 0.8321
Epoch 2/100
148/148 [==============================] - 0s 1ms/step - loss: 0.1548 - accuracy: 0.9522 - val_loss: 0.7513 - val_accuracy: 0.7458
Epoch 3/100
148/148 [==============================] - 0s 1ms/step - loss: 0.1878 - accuracy: 0.9396 - val_loss: 0.1235 - val_accuracy: 0.9538
Epoch 4/100
148/148 [===================

148/148 [==============================] - 0s 2ms/step - loss: 0.0080 - accuracy: 0.9984 - val_loss: 0.0379 - val_accuracy: 0.9884
Epoch 21/100
148/148 [==============================] - 0s 2ms/step - loss: 0.0038 - accuracy: 0.9993 - val_loss: 0.0708 - val_accuracy: 0.9769
Epoch 22/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0087 - accuracy: 0.9967 - val_loss: 0.0437 - val_accuracy: 0.9844
Epoch 23/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0368 - val_accuracy: 0.9891
Epoch 24/100
148/148 [==============================] - 0s 2ms/step - loss: 0.0013 - accuracy: 0.9999 - val_loss: 0.0439 - val_accuracy: 0.9884
Epoch 25/100
148/148 [==============================] - 1s 4ms/step - loss: 0.0098 - accuracy: 0.9973 - val_loss: 0.1060 - val_accuracy: 0.9762
Epoch 26/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0149 - accuracy: 0.9952 - val_loss: 0.0546 - val_accuracy: 0.9816
Epoch

Epoch 11/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0294 - accuracy: 0.9891 - val_loss: 0.0569 - val_accuracy: 0.9762
Epoch 12/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0282 - accuracy: 0.9900 - val_loss: 0.0765 - val_accuracy: 0.9735
Epoch 13/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0220 - accuracy: 0.9929 - val_loss: 0.0562 - val_accuracy: 0.9769
Epoch 14/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0238 - accuracy: 0.9922 - val_loss: 0.0645 - val_accuracy: 0.9735
Epoch 15/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0282 - accuracy: 0.9900 - val_loss: 0.0520 - val_accuracy: 0.9823
Epoch 16/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0172 - accuracy: 0.9942 - val_loss: 0.0610 - val_accuracy: 0.9742
Epoch 17/100
148/148 [==============================] - 0s 1ms/step - loss: 0.0195 - accuracy: 0.9937 - val_loss: 0.0526 - val_accuracy:

RandomizedSearchCV(estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fabd0558340>,
                   param_distributions={'n_layers': [1, 2, 3],
                                        'n_neurons': [17, 19]})

In [21]:
# Get best parameters
rnd_clf.best_params_

{'n_neurons': 19, 'n_layers': 1}

In [22]:
# Check best score
rnd_clf.best_score_

0.982995069026947

## Visualize training graphs

In [23]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6011

Reusing TensorBoard on port 6011 (pid 38064), started 1 day, 9:56:17 ago. (Use '!kill 38064' to kill it.)

## Saving the model

In [24]:
# Create new model with best_params

model_saved = build_model(n_neurons=rnd_clf.best_params_['n_neurons'], 
                     n_layers=rnd_clf.best_params_['n_layers'],
                     activation='elu',
                     optimizer='Nadam')

In [27]:
# Train model

model_saved.fit(X_train, y_train.toarray(), epochs=400, callbacks=tensorboard_cb)

Epoch 1/400
184/184 [==============================] - 0s 1ms/step - loss: 0.0369 - accuracy: 0.9922
Epoch 2/400
184/184 [==============================] - 0s 870us/step - loss: 0.0106 - accuracy: 0.9968
Epoch 3/400
184/184 [==============================] - 0s 868us/step - loss: 9.1653e-04 - accuracy: 0.9998
Epoch 4/400
184/184 [==============================] - 0s 904us/step - loss: 4.6517e-04 - accuracy: 1.0000
Epoch 5/400
184/184 [==============================] - 0s 918us/step - loss: 2.5511e-04 - accuracy: 1.0000
Epoch 6/400
184/184 [==============================] - 0s 1ms/step - loss: 2.1738e-04 - accuracy: 1.0000
Epoch 7/400
184/184 [==============================] - 0s 1ms/step - loss: 1.9168e-04 - accuracy: 1.0000
Epoch 8/400
184/184 [==============================] - 0s 895us/step - loss: 1.8030e-04 - accuracy: 1.0000
Epoch 9/400
184/184 [==============================] - 0s 882us/step - loss: 1.5951e-04 - accuracy: 1.0000
Epoch 10/400
184/184 [=============================

184/184 [==============================] - 0s 879us/step - loss: 1.8184e-05 - accuracy: 1.0000
Epoch 78/400
184/184 [==============================] - 0s 857us/step - loss: 1.7401e-05 - accuracy: 1.0000
Epoch 79/400
184/184 [==============================] - 0s 852us/step - loss: 1.6450e-05 - accuracy: 1.0000
Epoch 80/400
184/184 [==============================] - 0s 838us/step - loss: 1.5599e-05 - accuracy: 1.0000
Epoch 81/400
184/184 [==============================] - 0s 849us/step - loss: 1.4767e-05 - accuracy: 1.0000
Epoch 82/400
184/184 [==============================] - 0s 846us/step - loss: 1.3970e-05 - accuracy: 1.0000
Epoch 83/400
184/184 [==============================] - 0s 836us/step - loss: 1.3103e-05 - accuracy: 1.0000
Epoch 84/400
184/184 [==============================] - 0s 845us/step - loss: 1.2554e-05 - accuracy: 1.0000
Epoch 85/400
184/184 [==============================] - 0s 834us/step - loss: 1.1885e-05 - accuracy: 1.0000
Epoch 86/400
184/184 [===================

184/184 [==============================] - 0s 885us/step - loss: 6.3191e-06 - accuracy: 1.0000
Epoch 153/400
184/184 [==============================] - 0s 857us/step - loss: 6.0142e-06 - accuracy: 1.0000
Epoch 154/400
184/184 [==============================] - 0s 838us/step - loss: 5.6709e-06 - accuracy: 1.0000
Epoch 155/400
184/184 [==============================] - 0s 840us/step - loss: 5.3508e-06 - accuracy: 1.0000
Epoch 156/400
184/184 [==============================] - 0s 843us/step - loss: 5.0641e-06 - accuracy: 1.0000
Epoch 157/400
184/184 [==============================] - 0s 864us/step - loss: 4.8369e-06 - accuracy: 1.0000
Epoch 158/400
184/184 [==============================] - 0s 862us/step - loss: 4.6064e-06 - accuracy: 1.0000
Epoch 159/400
184/184 [==============================] - 0s 849us/step - loss: 4.4143e-06 - accuracy: 1.0000
Epoch 160/400
184/184 [==============================] - 0s 847us/step - loss: 4.1664e-06 - accuracy: 1.0000
Epoch 161/400
184/184 [==========

184/184 [==============================] - 0s 933us/step - loss: 8.3736e-08 - accuracy: 1.0000
Epoch 228/400
184/184 [==============================] - 0s 938us/step - loss: 7.9885e-08 - accuracy: 1.0000
Epoch 229/400
184/184 [==============================] - 0s 960us/step - loss: 7.1392e-08 - accuracy: 1.0000
Epoch 230/400
184/184 [==============================] - 0s 1ms/step - loss: 6.6000e-08 - accuracy: 1.0000
Epoch 231/400
184/184 [==============================] - 0s 946us/step - loss: 6.4986e-08 - accuracy: 1.0000
Epoch 232/400
184/184 [==============================] - 0s 930us/step - loss: 5.7730e-08 - accuracy: 1.0000
Epoch 233/400
184/184 [==============================] - 0s 955us/step - loss: 5.5905e-08 - accuracy: 1.0000
Epoch 234/400
184/184 [==============================] - 0s 877us/step - loss: 5.6655e-08 - accuracy: 1.0000
Epoch 235/400
184/184 [==============================] - 0s 934us/step - loss: 4.7331e-08 - accuracy: 1.0000
Epoch 236/400
184/184 [============

184/184 [==============================] - 0s 931us/step - loss: 1.4392e-09 - accuracy: 1.0000
Epoch 303/400
184/184 [==============================] - 0s 860us/step - loss: 1.2770e-09 - accuracy: 1.0000
Epoch 304/400
184/184 [==============================] - 0s 836us/step - loss: 1.4189e-09 - accuracy: 1.0000
Epoch 305/400
184/184 [==============================] - 0s 853us/step - loss: 1.3176e-09 - accuracy: 1.0000
Epoch 306/400
184/184 [==============================] - 0s 841us/step - loss: 1.3581e-09 - accuracy: 1.0000
Epoch 307/400
184/184 [==============================] - 0s 852us/step - loss: 1.1351e-09 - accuracy: 1.0000
Epoch 308/400
184/184 [==============================] - 0s 843us/step - loss: 1.3176e-09 - accuracy: 1.0000
Epoch 309/400
184/184 [==============================] - 0s 832us/step - loss: 1.3176e-09 - accuracy: 1.0000
Epoch 310/400
184/184 [==============================] - 0s 855us/step - loss: 1.2973e-09 - accuracy: 1.0000
Epoch 311/400
184/184 [==========

184/184 [==============================] - 0s 870us/step - loss: 9.7297e-10 - accuracy: 1.0000
Epoch 378/400
184/184 [==============================] - 0s 835us/step - loss: 1.0946e-09 - accuracy: 1.0000
Epoch 379/400
184/184 [==============================] - 0s 846us/step - loss: 1.0541e-09 - accuracy: 1.0000
Epoch 380/400
184/184 [==============================] - 0s 835us/step - loss: 1.0946e-09 - accuracy: 1.0000
Epoch 381/400
184/184 [==============================] - 0s 848us/step - loss: 9.7297e-10 - accuracy: 1.0000
Epoch 382/400
184/184 [==============================] - 0s 828us/step - loss: 9.5270e-10 - accuracy: 1.0000
Epoch 383/400
184/184 [==============================] - 0s 861us/step - loss: 9.3243e-10 - accuracy: 1.0000
Epoch 384/400
184/184 [==============================] - 0s 918us/step - loss: 8.9189e-10 - accuracy: 1.0000
Epoch 385/400
184/184 [==============================] - 0s 894us/step - loss: 1.0541e-09 - accuracy: 1.0000
Epoch 386/400
184/184 [==========

In [28]:
%reload_ext tensorboard
%tensorboard --logdir=./my_logs --port=6011

Reusing TensorBoard on port 6011 (pid 38064), started 1 day, 10:03:45 ago. (Use '!kill 38064' to kill it.)

In [30]:
# Intermediate evaluation:

model_saved.evaluate(X_test, y_test.toarray(), verbose=1)

46/46 [==============================] - 0s 785us/step - loss: 0.0767 - accuracy: 0.9905


[0.07673197239637375, 0.9904826879501343]

In [32]:
# Save model

config = model_saved.get_config()
keras.models.save_model(model=model_saved,
                        filepath='Motion_detector.h5')

In [33]:
# Restore model:

model_f = keras.models.Sequential.from_config(config)
model_f.compile(loss="categorical_crossentropy", metrics="accuracy", optimizer='Nadam')

In [41]:
model_f.load_weights('Motion_detector.h5')
model_f.set_weights()
model_f.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_31 (Flatten)         (None, 562)               0         
_________________________________________________________________
dense_92 (Dense)             (None, 19)                10697     
_________________________________________________________________
dense_93 (Dense)             (None, 6)                 120       
Total params: 10,817
Trainable params: 10,817
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Verify correct compiling model:
np.testing.assert_allclose(
    model_saved.predict(X_test), model_f.predict(X_test))

In [43]:
# Restored model evaluation:
model_f.evaluate(X_val, y_val_encoded.toarray(), 
                   verbose=1, 
                   callbacks=[early_stopping_callback, tensorboard_cb])

93/93 [==============================] - 0s 631us/step - loss: 0.5409 - accuracy: 0.9501


[0.5408585667610168, 0.9501187801361084]

In [38]:
# See how many predictions of "WALKING" in the first 1000 rows

prediction = model_f.predict(X_val)
res = list(ohe.inverse_transform(prediction)[0:1000])
res.count('WALKING')

168

In [39]:
# See how many true 'WALKING' label are in the first 100o rows

list(ohe.inverse_transform(y_val_encoded[0:1000])).count('WALKING')

171

### Without any further training, the model can be restored and provide 95% accuracy, or can be used for transfer learning 